In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'



import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

plt.style.use('bmh')
sns.set_style('darkgrid')

### Section 1: Problem Statement

The objective of this task is to predict the **price of diamonds** using the **K-Nearest Neighbors (KNN) Regression** algorithm.

The dataset contains a mix of **numerical and categorical features**, and the target variable (`price`) is continuous, making this a **regression problem**.

The core requirement of this task is to implement the **KNN algorithm from scratch**, while using sklearn utilities only for data splitting and preprocessing.


### Section 2: Load Dataset

In [2]:
# Load the dataset
df = pd.read_csv("diamonds.csv")

In [3]:
# Check shape and column names
df.shape
df.columns

(53940, 10)

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y',
       'z'],
      dtype='object')

In [4]:
# Display first few rows
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


#### Observations

- The dataset contains **53,940 rows and 10 columns**, which matches the dataset description provided.
- Column names are correctly loaded and include both **numerical and categorical features**.
- The data is successfully loaded without any structural issues.


### Section 3: Basic Data Understanding

In [5]:
# Dataset structure and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [6]:
# Statistical summary of numerical features
df.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


#### Observations

- The dataset has **no missing values**, which simplifies preprocessing.
- There are **3 categorical features** (`cut`, `color`, `clarity`) and **7 numerical features** including the target `price`.
- The target variable `price` shows a **wide range** and is **right-skewed**, indicating the presence of high-priced diamonds.
- Numerical features have **very different scales**, which is critical for distance-based algorithms like KNN.
- Features `x`, `y`, and `z` contain **zero values**, which are physically unrealistic but are retained as no data-cleaning step is specified in the task.


### Section 4: Define Features and Target

In [7]:
# Separate input features and target
X = df.drop(columns=["price"])
y_price = df["price"]

In [8]:
# Identify categorical and numerical columns
categorical_cols = ["cut", "color", "clarity"]
numerical_cols = [col for col in X.columns if col not in categorical_cols]

categorical_cols
numerical_cols

['cut', 'color', 'clarity']

['carat', 'depth', 'table', 'x', 'y', 'z']

#### Observations

- The target variable `price` has been separated from the feature set.
- Categorical and numerical features are clearly identified, which is necessary for applying different preprocessing techniques.
- The target variable is named `y_price` to avoid confusion with the feature column `y`.


### Section 5: Train–Test Split (75:25)

In [9]:
from sklearn.model_selection import train_test_split


In [10]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_price, test_size=0.25, random_state=42
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((40455, 9), (13485, 9), (40455,), (13485,))

#### Observations

- The dataset is split into **75% training data** and **25% testing data**, as recommended.
- A fixed `random_state` ensures reproducibility of results.
- The split is performed **before any preprocessing** to prevent data leakage.


### Section 6: Data Preprocessing

#### Step 6.1: Encode Categorical Features

In [11]:
from sklearn.preprocessing import OneHotEncoder


In [12]:
# Initialize OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
ohe

,categories,'auto'
,drop,None
,sparse_output,False
,dtype,<class 'numpy.float64'>
,handle_unknown,'ignore'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [13]:
# Fit only on training data
X_train_cat = ohe.fit_transform(X_train[categorical_cols])
X_train_cat

array([[0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
# Transform test data
X_test_cat = ohe.transform(X_test[categorical_cols])
X_test_cat

array([[0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
X_train_cat.shape, X_test_cat.shape

((40455, 20), (13485, 20))

#### Observations

- Categorical features (`cut`, `color`, `clarity`) are converted into numerical form using **One-Hot Encoding**.
- The encoder is **fit only on training data** to avoid data leakage.
- `handle_unknown="ignore"` ensures stability if unseen categories appear in test data.
- One-hot encoding is necessary because KNN relies on distance calculations.


#### Step 6.2: Scale Numerical Features

In [16]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler
scaler = StandardScaler()
scaler

,copy,True
,with_mean,True
,with_std,True


In [17]:
# Fit on training numerical data
X_train_num = scaler.fit_transform(X_train[numerical_cols])
X_train_num

array([[-1.15666465,  2.20783668,  0.24241403, -1.58998506, -1.54444639,
        -1.36581585],
       [ 0.08691672,  0.03851691, -0.65492279,  0.27356006,  0.29150568,
         0.28214948],
       [ 0.52954737, -0.4513295 ,  0.24241403,  0.73721722,  0.67618135,
         0.63427882],
       ...,
       [-0.98804345, -1.01115395,  0.24241403, -1.10849493, -1.11605757,
        -1.18270859],
       [ 0.21338262,  0.73829748,  0.69108244,  0.35380841,  0.25653516,
         0.39483087],
       [ 0.71924623, -0.9411759 ,  0.24241403,  0.9690458 ,  0.92097496,
         0.80330091]])

In [18]:
# Transform test numerical data
X_test_num = scaler.transform(X_test[numerical_cols])
X_test_num

array([[-1.1777423 ,  0.24845108, -0.65492279, -1.57215209, -1.5182185 ,
        -1.50666759],
       [-0.46110219, -1.22108813, -0.20625438, -0.26142897, -0.2767652 ,
        -0.39393886],
       [-0.8404999 ,  0.24845108, -1.1035912 , -0.86774987, -0.87126397,
        -0.83057925],
       ...,
       [ 0.86678978,  0.45838525, -1.55225961,  1.00471173,  0.92971759,
         1.01457852],
       [ 0.44523677,  0.31842913,  0.69108244,  0.60346996,  0.62372557,
         0.648364  ],
       [ 0.50846972,  0.73829748,  0.69108244,  0.65696887,  0.59749769,
         0.71878987]])

In [19]:
X_train_num.shape
X_test_num.shape

(40455, 6)

(13485, 6)

#### Observations

- Numerical features are scaled using **StandardScaler**.
- Feature scaling is essential for KNN because distance calculations are sensitive to magnitude differences.
- The scaler is fitted only on training data to prevent information leakage.
- This step ensures all numerical features contribute fairly to distance computation.


#### Step 6.3: Combine Processed Features

In [20]:
X_train_processed = np.hstack((X_train_num, X_train_cat))
X_test_processed = np.hstack((X_test_num, X_test_cat))

X_train_processed.shape, X_test_processed.shape

((40455, 26), (13485, 26))

#### Observations

- Numerical and categorical features are combined into a single feature matrix.
- The final processed dataset is fully numerical and ready for KNN implementation.
- Both training and testing sets have consistent feature dimensions.


### Section 7: Prepare Data for Scratch KNN

In [21]:
# Convert preprocessed features and target to NumPy arrays
X_train_np = X_train_processed
X_test_np = X_test_processed

y_train_np = y_train.values
y_test_np = y_test.values

# Check shapes
X_train_np.shape, X_test_np.shape, y_train_np.shape, y_test_np.shape


((40455, 26), (13485, 26), (40455,), (13485,))

In [22]:
# Create smaller subsets to reduce computational cost
subset_size = 2000

X_train_knn = X_train_np[:subset_size]
y_train_knn = y_train_np[:subset_size]

X_test_knn = X_test_np[:500]
y_test_knn = y_test_np[:500]

# Verify subset shapes
X_train_knn.shape, X_test_knn.shape


((2000, 26), (500, 26))

#### Observations

- After preprocessing, the dataset contains 26 numerical features for each sample.
- Training and test sets have consistent feature dimensions, ensuring valid distance computation.
- Target variables are converted to NumPy arrays for numerical operations.
- A smaller subset of the data is created only to reduce computational cost, as scratch KNN has quadratic time complexity.
- Subsetting does not affect the preprocessing logic or the validity of the model implementation.


### Section 8: KNN Regression – Scratch Implementation

- In this section, we implement the KNN regression algorithm from scratch. 
- The model predicts the price of a diamond by averaging the prices of its K nearest neighbors based on Euclidean distance.

#### Step 8.1: Euclidean Distance Function

In [23]:
def euclidean_distance(point1, point2):
    """
    Compute Euclidean distance between two points.
    """
    return np.sqrt(np.sum((point1 - point2) ** 2))


#### Observations

- Euclidean distance is used as the similarity measure for KNN.
- The function computes distance between two feature vectors of equal length.
- This distance metric is appropriate because all features are scaled.


#### Step 8.2: Predict Price for a Single Test Point

In [24]:
def knn_predict_single(X_train, y_train, test_point, k):
    """
    Predict the target value for a single test point using KNN regression.
    """
    distances = []

    # Compute distance from test point to each training point
    for i in range(len(X_train)):
        dist = euclidean_distance(X_train[i], test_point)
        distances.append((dist, y_train[i]))

    # Sort by distance
    distances.sort(key=lambda x: x[0])

    # Select k nearest neighbors
    k_nearest = distances[:k]

    # Compute mean of k nearest neighbors' target values
    prediction = np.mean([value for _, value in k_nearest])

    return prediction


#### Observations

- Distance is computed between the test point and all training samples.
- Training samples are sorted based on distance.
- The prediction is calculated as the mean of the target values of the k nearest neighbors.
- This follows the standard KNN regression logic.


#### Step 8.3: Predict for All Test Points

In [25]:
def knn_predict(X_train, y_train, X_test, k):
    """
    Predict target values for all test points.
    """
    predictions = []

    for i in range(len(X_test)):
        pred = knn_predict_single(X_train, y_train, X_test[i], k)
        predictions.append(pred)

    return np.array(predictions)


#### Observations

- The function applies KNN prediction to each test sample individually.
- Predictions are stored and returned as a NumPy array.
- Simple loops are used for clarity and correctness.
- No external machine learning libraries are used in this implementation.


#### Step 8.4: Choose Value of K

In [26]:
# Choose number of neighbors
k = 5
k


5

#### Observations

- A small odd value of K is chosen to balance bias and variance.
- Lower K captures local patterns but may increase variance.
- The selected K is reasonable for demonstrating scratch implementation.


#### Step 8.5: Generate Predictions Using Scratch KNN

In [27]:
# Predict prices for test subset
y_pred_knn = knn_predict(X_train_knn, y_train_knn, X_test_knn, k)

y_pred_knn[:10]


array([ 825.2, 2319.4, 1046.2, 1204. , 6269.4, 2365.8, 1899.2, 1708. ,
       1672. , 4645.2])

#### Observations

- Scratch KNN successfully generates predictions for test samples.
- The output is a continuous numerical value, suitable for regression.
- Predictions are based only on training subset data, ensuring no leakage.


### Section 9: Prediction on Test Data

- In this section, we use the scratch KNN model implemented earlier to generate predictions on the test subset.

#### Step 9.1: Predict Diamond Prices on Test Data

In [28]:
# Generate predictions using scratch KNN on test subset
y_test_predictions = knn_predict(
    X_train_knn,
    y_train_knn,
    X_test_knn,
    k
)

# Display first few predictions
y_test_predictions[:10]


array([ 825.2, 2319.4, 1046.2, 1204. , 6269.4, 2365.8, 1899.2, 1708. ,
       1672. , 4645.2])

#### Observations

- Predictions are generated using the scratch KNN regression model.
- The output consists of continuous numerical values, representing predicted diamond prices.
- Predictions are computed only for the test subset to keep computation manageable.
- The training data used for prediction is strictly separated from the test data, ensuring no data leakage.


#### Step 9.2: Compare Actual vs Predicted Values (Quick Sanity Check)

In [29]:
# Compare actual and predicted prices for first few test samples
comparison_df = pd.DataFrame({
    "Actual Price": y_test_knn[:10],
    "Predicted Price": y_test_predictions[:10]
})

comparison_df


,Actual Price,Predicted Price
0,559,825.2
1,2201,2319.4
2,1238,1046.2
3,1304,1204.0
4,6901,6269.4
5,3011,2365.8
6,1765,1899.2
7,1679,1708.0
8,2102,1672.0
9,4789,4645.2


#### Observations

- The scratch KNN model produces **continuous-valued predictions**, confirming correct regression behavior.
- Predicted prices are **reasonably close to actual prices**, especially for mid-range diamonds.
- The model tends to **smooth extreme values**, as predictions are averages of neighboring prices.
- Larger deviations are observed for higher-priced diamonds, which is expected due to price skewness.
- Overall, predictions follow the **general trend of actual prices**, indicating the scratch implementation is functioning correctly.


### Section 10: Model Evaluation

- In this section, the performance of the scratch KNN regression model is evaluated using standard regression metrics.

#### Step 10.1: Import Evaluation Metrics

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


#### Step 10.2: Compute Evaluation Metrics

In [31]:
# Calculate evaluation metrics
mae = mean_absolute_error(y_test_knn, y_test_predictions)
rmse = np.sqrt(mean_squared_error(y_test_knn, y_test_predictions))
r2 = r2_score(y_test_knn, y_test_predictions)

print(mae)
print(rmse)
print(r2)


675.5628
1213.6907569558236
0.9209710302427896


#### Observations

- **Mean Absolute Error (MAE):**  
  Represents the average absolute difference between actual and predicted diamond prices.  
  It indicates how much, on average, the prediction deviates from the true price in dollar terms.

- **Root Mean Squared Error (RMSE):**  
  Penalizes larger errors more heavily than MAE and highlights cases where predictions are far from actual prices.  
  Useful for understanding the impact of large pricing mistakes.

- **R² Score:**  
  Measures how well the model explains the variance in diamond prices.  
  A higher R² value indicates better predictive performance of the KNN model.


### Section 11: Sklearn KNN Regression (Comparison Model)

- In this section, a KNN regression model using sklearn is trained and evaluated.
- This model is used only for comparison with the scratch implementation.

#### Step 11.1: Import Sklearn KNN Regressor

In [32]:
from sklearn.neighbors import KNeighborsRegressor


#### Step 11.2: Train Sklearn KNN Model

In [33]:
# Initialize sklearn KNN regressor with same K
knn_sklearn = KNeighborsRegressor(n_neighbors=k, metric='euclidean')
knn_sklearn

# Train model on the same training subset
knn_sklearn.fit(X_train_knn, y_train_knn)


,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'euclidean'
,metric_params,None
,n_jobs,None


,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'euclidean'
,metric_params,None
,n_jobs,None


#### Step 11.3: Predict on Test Data

In [34]:
# Generate predictions using sklearn KNN
y_pred_sklearn = knn_sklearn.predict(X_test_knn)

# Display first few predictions
y_pred_sklearn[:10]


array([ 825.2, 2319.4, 1046.2, 1204. , 6269.4, 2365.8, 1899.2, 1708. ,
       1672. , 4645.2])

#### Step 11.4: Evaluate Sklearn KNN Model

In [35]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate evaluation metrics for sklearn KNN
mae_sk = mean_absolute_error(y_test_knn, y_pred_sklearn)

mse_sk = mean_squared_error(y_test_knn, y_pred_sklearn)
rmse_sk = np.sqrt(mse_sk)

r2_sk = r2_score(y_test_knn, y_pred_sklearn)

print(mae_sk)
print(rmse_sk)
print(r2_sk)


675.5628
1213.6907569558236
0.9209710302427896


#### Observations

- Evaluation metrics are computed using the **same test subset** as the scratch KNN model.
- Using identical evaluation criteria ensures a **fair and direct comparison**.
- These metrics quantify the prediction error and explanatory power of the sklearn KNN model.


### Section 12: Comparison & Final Observations

- In this section, the performance of the scratch KNN regression model is compared with the sklearn KNN regression model using the same evaluation metrics.

#### Step 12.1: Metric Comparison

In [36]:
print("---- Model Comparison (Scratch vs Sklearn) ----")
print("MAE  :", mae, "vs", mae_sk)
print("RMSE :", rmse, "vs", rmse_sk)
print("R²   :", r2, "vs", r2_sk)


---- Model Comparison (Scratch vs Sklearn) ----
MAE  : 675.5628 vs 675.5628
RMSE : 1213.6907569558236 vs 1213.6907569558236
R²   : 0.9209710302427896 vs 0.9209710302427896


#### Observations

- The scratch KNN model and sklearn KNN model produce **very similar evaluation metrics**.
- This similarity is expected because both models use:
  - Euclidean distance
  - The same value of K
  - Uniform weighting of neighbors
- Sklearn’s `KNeighborsRegressor` with default `weights='uniform'` predicts by averaging the target values of the nearest neighbors, which matches the scratch implementation logic.
- Minor differences, if any, can arise from numerical precision and implementation-level optimizations.
- The close match confirms that the **scratch KNN implementation is correct and reliable**.


### Section 13: Limitations

### Section 13: Limitations

- **High computational cost:**  
  The scratch KNN implementation computes distances between each test point and all training points, resulting in high time complexity. This makes it impractical for large datasets without optimization.

- **Memory intensive:**  
  KNN requires storing the entire training dataset in memory, as no explicit training phase reduces the data size.

- **Scalability issues:**  
  Due to quadratic time complexity, a subset of the dataset was used for model training and evaluation. This limitation highlights why KNN does not scale well to very large datasets.

- **Sensitivity to noisy and unrealistic values:**  
  The presence of zero values in physical dimensions (`x`, `y`, `z`) can distort distance calculations and negatively impact predictions.

- **Dependence on feature scaling:**  
  KNN performance is highly sensitive to feature magnitudes. Without proper scaling, distance-based predictions would be biased toward features with larger numeric ranges.

- **Choice of K:**  
  Model performance depends on the selected value of K. A poor choice can lead to underfitting (large K) or overfitting (small K).
